# Network Blending Experiment

## Setup Google Drive Connection

In [1]:
# Check GPU connection
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-7ce30f3f-266a-4b28-7cf3-fbc0f4bd9589)


In [2]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
!pip install gdown --upgrade

if os.path.isdir("/content/drive/MyDrive/stylegan3-fun-blend"):
    %cd "/content/drive/MyDrive/stylegan3-fun-blend"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !git clone https://github.com/adamdavidcole/stylegan3-fun-blend.git
    %cd stylegan3-fun-blend
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    # !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/adamdavidcole/stylegan3-fun-blend.git
    %cd stylegan3-fun-blend
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/stylegan3-fun-blend


In [5]:
# Update all code files in drive repo 
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- "*.py" 
!git checkout origin/main -- "*.ipynb"

Already up to date.
Saved working directory and index state WIP on main: 59cff72 drive+collab test3


In [4]:
!pip install einops ninja gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 22.7 MB/s 


In [39]:
import numpy as np
from datetime import datetime

## Network Setup

In [67]:
network_key = "butterfly"
cfg="stylegan3-t"
pretrained_network = "ffhqu256"
dataset = "/content/drive/MyDrive/stylegan3/datasets/butterflys_256_2-256x256.zip"

results_outdir = f"./results/{network_key}"

if not os.path.isdir(outdir):
    !mkdir -p $outdir

## Training

In [13]:
!python train.py --outdir=$outdir --cfg=$cfg --data=$dataset \
    --gpus=1 --batch=32 --batch-gpu=16  --gamma=6.6 --mirror=1 --kimg=50 --snap=1 --tick=1 --img-snap=1 --cbase=16384 \
    --resume=ffhqu256

Creating output directory...

Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2,
      "freeze_layers": 0,
      "freeze_embed": false
    },
    "channel_base": 32768,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9988915792636801,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
   

## Projection

In [ ]:
network_source_folder = "00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256"

In [27]:
# Upload File 
outdir = "projection_source_images"
if not os.path.isdir(outdir):
  !mkdir -p $outdir

def upload_files():
  filepaths = []
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    filepath = f"{outdir}/{k}"
    open(filepath, 'wb').write(v)
    filepaths.append(filepath)
  return list(filepaths)

uploaded = upload_files();
print(uploaded)

Saving self-image.jpeg to self-image (2).jpeg
['projection_source_images/self-image.jpeg']


In [ ]:
# Project Image
uploaded_file = uploaded[0]
print(uploaded_file)

!python projector.py --target=$uploaded_file --project-in-wplus --save-video --num-steps=5000 --stabilize-projection \
       --cfg=stylegan3-r --network=ffhqu256

projection_source_images/self-image.jpeg
Loading networks from "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Projecting in W+ latent space...
Starting from W midpoint using 10000 samples...
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


## Generation

In [74]:
most_recent_training_result = os.listdir(results_outdir)[-1]
print(most_recent_training_result)

path_to_most_recent_training_result = f"{results_outdir}/{most_recent_training_result}"
training_checkpoints = [f"{path_to_most_recent_training_result}/{f}" for f in os.listdir(path_to_most_recent_training_result) if f.endswith('.pkl')]
print(training_checkpoints)


00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256
['./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000000.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000001.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000002.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000003.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000004.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000005.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot

In [66]:
projected_w_path = "out/projection/00004-projection-wplus-wavgstart-sgan2/projected_wplus_wavg_final.npy"
result_name="test_result"
# projection_network_pkl = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"];
# projection_network_pkl = 

projection_outdir=f"{outdir}/projections"

print(projection_outdir)

if not os.path.isdir(projection_outdir):
  !mkdir -p $projection_outdir

projection_source_images/projections


In [54]:
# Generate image from projection
projected_w = np.load(projected_w_path)
print(projected_w.shape)

(1, 16, 512)


In [75]:
# Generate N images across network blend
import torch
import dnnlib
from dnnlib.util import format_time
import legacy
import PIL.Image

from torch_utils import gen_utils


# projection_network_pkl = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"]


def gen_img_from_network(network_pkl_path):
  network_pkl_name = network_pkl_path.split('/')[-1]
  print('Loading networks from "%s"...' % network_pkl_path)
  device = torch.device('cuda')
  with dnnlib.util.open_url(network_pkl_path) as fp:
      G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

  projected_w_tensor = torch.tensor(projected_w).to(device)
  synth_image = gen_utils.w_to_img(G, dlatents=projected_w_tensor, noise_mode='const')[0]
  PIL.Image.fromarray(synth_image, 'RGB').save(f'{projection_outdir}/{result_name}_{network_pkl_name}.jpg')

for i in range(10):
  gen_img_from_network(training_checkpoints[i])

Loading networks from "./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000000.pkl"...
Loading networks from "./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000001.pkl"...
Loading networks from "./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000002.pkl"...
Loading networks from "./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000003.pkl"...
Loading networks from "./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000004.pkl"...
Loading networks from "./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000005.pkl"...
Loading networks from "./results/butterfly/00005-stylegan3

## Selection

## Refinement